## Paso 1: Instalar las dependencias y la configuración del notebook en GPU

In [0]:
!pip install tensorflow-gpu==2.0.0.alpha0

## Paso 2: Importar las librerías necesarias

In [0]:
import tensorflow as tf

from tensorflow.keras.datasets import imdb

In [0]:
tf.__version__

'2.0.0-alpha0'

## Paso 3: Pre procesado de datos


### Configurar parámetros del dataset

In [0]:
number_of_words = 20000
max_len = 100

### Carga del dataset de IMDB

In [0]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=number_of_words)

### Cortar secuencias de texto de la misma longitud

In [0]:
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train, maxlen=max_len)

In [0]:
X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, maxlen=max_len)

### Configurar parámetros de la capa de Embedding

In [0]:
vocab_size = number_of_words
vocab_size

20000

In [0]:
embed_size = 128

## Paso 4: Construir la Red Neuronal Recurrente

### Definir el modelo

In [0]:
model = tf.keras.Sequential()

### Añadir la capa de embedding

In [0]:
model.add(tf.keras.layers.Embedding(vocab_size, embed_size, input_shape=(X_train.shape[1],)))

### Añadir la capa de LSTM

- unidades: 128
- función de activación: tanh

In [0]:
model.add(tf.keras.layers.LSTM(units=128, activation='tanh'))

W0510 22:26:40.657396 140526441928576 tf_logging.py:161] <tensorflow.python.keras.layers.recurrent.UnifiedLSTM object at 0x7fce6e733198>: Note that this layer is not optimized for performance. Please use tf.keras.layers.CuDNNLSTM for better performance on GPU.


### Añadir la capa totalmente conectada de salida

- unidades: 1
- función de activación: sigmoid

In [0]:
model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

### Compilar el modelo

In [0]:
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [0]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 100, 128)          2560000   
_________________________________________________________________
unified_lstm (UnifiedLSTM)   (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 2,691,713
Trainable params: 2,691,713
Non-trainable params: 0
_________________________________________________________________


### Entrenar el modelo

In [0]:
model.fit(X_train, y_train, epochs=3, batch_size=128)

Epoch 1/3
25000/25000 [==============================] - 4s 173us/sample - loss: 0.4640 - accuracy: 0.7792
Epoch 2/3
25000/25000 [==============================] - 2s 84us/sample - loss: 0.2884 - accuracy: 0.8842
Epoch 3/3
25000/25000 [==============================] - 2s 84us/sample - loss: 0.2291 - accuracy: 0.9129


### Evaluar el modelo

In [0]:
test_loss, test_acurracy = model.evaluate(X_test, y_test)

25000/25000 [==============================] - 3s 136us/sample - loss: 0.3403 - accuracy: 0.8528


In [0]:
print("Test accuracy: {}".format(test_acurracy))

Test accuracy: 0.8527600169181824
